# LSTM Time Series Forecasting Input / Output Shape

###  Univariate(단변수) Multi-step Input LSTM and Single-step Output
이 모델은 단변수 시계열 데이터에 대한 LSTM을 사용하여 여러 타임 스텝의 입력을 기반으로 단일 타임 스텝의 출력을 생성합니다. 단변수 시계열 데이터란 하나의 특성만을 가진 데이터를 의미합니다. 여기서, 모델은 과거의 일정한 시간 윈도우(window) 내 데이터를 사용하여 미래의 한 단계 값을 예측합니다. 예를 들어, 과거 30일 동안의 주식 가격을 기반으로 내일의 주식 가격을 예측할 수 있습니다.

### Multivariate(다변수) Multi-step Input LSTM and Single-step Output
이 모델은 다변수 시계열 데이터에 대한 LSTM을 사용하여 여러 타임 스텝의 입력을 기반으로 단일 타임 스텝의 출력을 생성합니다. 다변수 시계열 데이터란 여러 개의 특성을 가진 데이터를 의미합니다. 이 모델은 단변수 버전과 비슷하지만, 여러 특성을 사용하여 예측을 수행합니다. 예를 들어, 주식 시장에서 과거 30일 동안의 주가, 거래량, 시가총액 등의 정보를 사용하여 내일의 주식 가격을 예측할 수 있습니다.

여기서 **multi-step size**란 **window size**를 의미합니다.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential

## 1. Univariate Multi-step Input and Single-step output LSTM 

- 단일변수 multi-timestep 입력 단일 timestep 출력  

- input feature - 1, output unit - 1

    ex) 과거 3 일간 종가 입력 $\rightarrow$ 내일 주가 예상

In [3]:
def windowed_ds(series, window_size, batch_size, shuffle_buffer):    
    ds = tf.data.Dataset.from_tensor_slices(series)  # 주어진 시계열 데이터로부터 데이터셋을 생성합니다.
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)  # 데이터셋을 지정된 윈도우 크기만큼의 부분 시퀀스로 나눕니다.
    ds = ds.flat_map(lambda window: window.batch(window_size+1))  # 각 윈도우를 단일 배치로 플랫하게 만듭니다. 
    ds = ds.map(lambda window: (window[:-1], window[-1]))  # 각 윈도우에서 마지막 값을 제외한 나머지로 입력 데이터를 구성하고, 마지막 값을 레이블로 사용
    ds = ds.batch(batch_size).prefetch(1)  # 데이터를 배치 사이즈만큼 묶고, prefetch를 사용하여 학습 데이터를 미리 준비합니다.
    return ds

In [4]:
# 시퀀스 데이터 정의
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# 윈도우 크기, 배치 크기, 셔플 버퍼 크기 설정
window_size = 3
batch_size = 1

# windowed_ds 함수를 사용하여 데이터셋 생성
dataset = windowed_ds(raw_seq, window_size, batch_size, 10)
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [5]:
for x, y in dataset:
    print(x.numpy().shape, y.numpy().shape)
    print(x.numpy())
    print(y.numpy())
    break

(1, 3) (1,)
[[10 20 30]]
[40]


In [6]:
model = Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=[None, 1]),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 10451 (40.82 KB)
Trainable params: 10451 (40.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Train이 안된 상태에서 간단히 모델을 작동시켜 입출력의 shape을 확인해 봅니다. 훈련이 안된 상태이므로 당연히 yhat 값은 의미가 없습니다.

In [7]:
# 입력 시퀀스 정의
x_input = np.array([[70, 80, 90]])

# 입력 시퀀스를 모델이 예측할 수 있는 형태로 재구성
# 여기서는 (샘플 수, 시퀀스 길이, 특성 수)로 변환
x_input = x_input.reshape(1, -1, 1)

# 모델을 사용하여 예측 수행
# window_size만큼의 데이터로 예측을 수행하기 위해 x_input의 일부만 사용
yhat = model.predict(x_input[:window_size])

# 입력 데이터와 예측 결과의 차원, 모양 출력
print(x_input.shape, yhat.shape)
# 입력 데이터와 예측 결과 출력
print(x_input, yhat)

1/1 [==============================] - 0s 121ms/step
(1, 3, 1) (1, 1)
[[[70]
  [80]
  [90]]] [[7.5554323]]


## 2. Multivariate Multi-step Input and Single-step Output LSTM 

- 여러개의 변수를 multi-timestep 입력 $\rightarrow$ 단일 time-step 출력  

- input feature - n, output unit - 1

    ex) 주가, 환율 과거 3 일치 입력하여 다음날 주가(환율) 예측
    ```
    [[ 10,  15],
     [ 20,  25],
     [ 30,  35]]   --> [40]   
    ```

- input sequence 정의

In [8]:
# 주식 가격 배열을 생성
in_stock = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
# 외환 배열을 생성
in_forex = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])

# 'in_stock' 배열의 4번째 원소부터 끝까지 선택하여 새로운 시퀀스 생성
out_seq = in_stock[3:]

# 생성된 시퀀스를 출력
out_seq

array([40, 50, 60, 70, 80, 90])

- `[row, columns]` 구조로 변환하고 열을 수평으로 쌓습니다.

여기서 in_stock과 in_forex 두 배열을 각각 2차원으로 변환하고, np.hstack() 함수를 사용하여 두 배열을 수평으로 연결합니다. 결과적으로 raw_seq는 두 배열을 나란히 놓은 형태의 2차원 배열이 됩니다. 

In [9]:
in_stock = in_stock.reshape(-1, 1)
in_forex = in_forex.reshape(-1, 1)

raw_seq = np.hstack((in_stock, in_forex))
raw_seq

array([[10, 15],
       [20, 25],
       [30, 35],
       [40, 45],
       [50, 55],
       [60, 65],
       [70, 75],
       [80, 85],
       [90, 95]])

ds.map()을 사용하여 입력 특성과 레이블로 구성된 훈련 샘플을 생성합니다. 여기서 window[:-1]는 마지막 단계를 제외한 모든 단계의 모든 feature를 사용하여 입력 특성을 구성하고, window[-1, 0]는 마지막 단계의 첫 번째 feature(예: 주식 가격)을 레이블로 사용합니다.

In [10]:
def windowed_ds(series, window_size, batch_size, shuffle_buffer):    
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size+1))
    ds = ds.map(lambda window: (window[:-1], window[-1, 0]))
    ds = ds.batch(batch_size).prefetch(1)
    return ds

In [11]:
window_size = 3
batch_size = 1
dataset = windowed_ds(raw_seq, window_size, batch_size, 10)
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 2), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [12]:
for x, y in dataset:
    print(x.numpy().shape, y.numpy().shape)
    print(x.numpy(), y.numpy())
    break

(1, 3, 2) (1,)
[[[10 15]
  [20 25]
  [30 35]]] [40]


In [13]:
model = Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=[None, 2]),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 50)                10600     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10651 (41.61 KB)
Trainable params: 10651 (41.61 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# 입력 데이터를 numpy 배열로 생성합니다. 여기서는 2개의 값을 가진 배열을 생성
x_input = np.array([[70, 80]])

# 입력 데이터를 모델이 예측에 사용할 수 있는 형태로 재구성합니다.
# 이 경우, 배치 크기, 시퀀스 길이, 특성 수를 나타내는 3차원 배열로 변경
# 여기서는 배치 크기를 1, 시퀀스 길이를 동적으로 설정(-1을 사용하여 자동 계산), 특성 수를 2로 설정
x_input = x_input.reshape(1, -1, 2)

# 모델을 사용하여 입력 데이터에 대한 예측을 수행합
# 여기서는 window_size 변수를 사용하여 입력 배열의 일부를 예측에 사용합니다.
yhat = model.predict(x_input[:window_size])

# 입력 데이터와 예측 결과의 형태를 출력합니다.
print(x_input.shape, yhat.shape)

1/1 [==============================] - 0s 14ms/step
(1, 1, 2) (1, 1)


### 다변수를 이용한 주가 예측

In [18]:
import yfinance as yf

df = yf.download('AAPL', start='2020-01-01', end='2023-12-31', progress=False)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,73.152641,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.441452,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,73.018692,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.675278,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.844353,132079200


In [22]:
dataset = df.iloc[:, [3, 5]].values
dataset.shape

(1006, 2)

In [20]:
window_size = 3
batch_size = 1

ds = tf.expand_dims(dataset, axis=1)
ds = tf.data.Dataset.from_tensor_slices(dataset)
ds = ds.window(window_size+1, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w: w.batch(window_size+1))
ds = ds.map(lambda w: (w[:-1], w[-1][0]))
ds = ds.batch(batch_size).prefetch(1)
ds

for x, y in ds:
    print(x.shape, y.shape)
    print()
    print(x.numpy(), y.numpy())
    break

(1, 3, 2) (1,)

[[[7.50875015e+01 1.35480400e+08]
  [7.43574982e+01 1.46322800e+08]
  [7.49499969e+01 1.18387200e+08]]] [74.59750366]
